In [ ]:
%pip install datasets evaluate transformers[sentencepiece]
%pip install accelerate -U
%pip install scikit-learn

In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_datasets = load_dataset("glue", "sst2")

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [3]:
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments("sst2-trainer", evaluation_strategy="epoch")

In [5]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "sst2")
    logits, lables = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=lables)

In [6]:
from transformers import Trainer

trainer = Trainer(
    model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [7]:
trainer.train()

  2%|▏         | 500/25257 [00:55<43:56,  9.39it/s] 

{'loss': 0.4356, 'learning_rate': 4.9010175396919665e-05, 'epoch': 0.06}


  4%|▍         | 1000/25257 [01:58<44:04,  9.17it/s]  

{'loss': 0.354, 'learning_rate': 4.8020350793839334e-05, 'epoch': 0.12}


  6%|▌         | 1500/25257 [03:10<46:00,  8.61it/s]   

...

 99%|█████████▉| 25000/25257 [52:06<00:26,  9.63it/s]

{'loss': 0.0935, 'learning_rate': 5.087698459832917e-07, 'epoch': 2.97}


                                                     
100%|██████████| 25257/25257 [52:45<00:00,  7.98it/s]


{'eval_loss': 0.40466275811195374, 'eval_accuracy': 0.908256880733945, 'eval_runtime': 5.3737, 'eval_samples_per_second': 162.272, 'eval_steps_per_second': 20.284, 'epoch': 3.0}
{'train_runtime': 3165.7141, 'train_samples_per_second': 63.824, 'train_steps_per_second': 7.978, 'train_loss': 0.20040190379882117, 'epoch': 3.0}


TrainOutput(global_step=25257, training_loss=0.20040190379882117, metrics={'train_runtime': 3165.7141, 'train_samples_per_second': 63.824, 'train_steps_per_second': 7.978, 'train_loss': 0.20040190379882117, 'epoch': 3.0})

## Model test

In [52]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_datasets = load_dataset("glue", "sst2")

tokenized_dataset = tokenizer(raw_datasets["validation"]["sentence"][:8], padding=True, return_tensors="pt")

In [53]:
model = AutoModelForSequenceClassification.from_pretrained("sst2-trainer\checkpoint-25000")

In [59]:
output = model(**tokenized_dataset)
print(raw_datasets["validation"].features)
raw_datasets["validation"]["sentence"][:8]

{'sentence': Value(dtype='string', id=None), 'label': ClassLabel(names=['negative', 'positive'], id=None), 'idx': Value(dtype='int32', id=None)}


["it 's a charming and often affecting journey . ",
 'unflinchingly bleak and desperate ',
 'allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . ',
 "the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . ",
 "it 's slow -- very , very slow . ",
 'although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women . ',
 'a sometimes tedious film . ',
 "or doing last year 's taxes with your ex-wife . "]

In [60]:
import torch

print(raw_datasets["validation"].features["label"])
print(raw_datasets["validation"]["label"][:8])

predictions = torch.nn.functional.softmax(output.logits, dim=-1)
print(predictions)


ClassLabel(names=['negative', 'positive'], id=None)
[1, 0, 1, 1, 0, 1, 0, 0]
tensor([[5.1018e-04, 9.9949e-01],
        [9.9901e-01, 9.9054e-04],
        [9.0881e-04, 9.9909e-01],
        [9.3941e-01, 6.0594e-02],
        [9.9918e-01, 8.1777e-04],
        [5.1939e-04, 9.9948e-01],
        [9.9921e-01, 7.8971e-04],
        [9.8606e-01, 1.3935e-02]], grad_fn=<SoftmaxBackward0>)
